In [1]:
import ipywidgets as widgets
from ipywidgets import interactive_output, interact
from menu import init_widgets, style, setup_menu, select_table
from dashboard import (
    Seaborn_Barplot,
    Seaborn_Boxplot,
    Seaborn_Scatter,
    Boxplot_ggplot,
    Histogram_ggplot,
)
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
from itables import init_notebook_mode, show

In [2]:
%load_ext sql

%sql duckdb:///../data/database/car_data.duckdb

# %config SqlMagic = False

In [3]:
years = %sql select DISTINCT(model_year) from all_vehicles
years = tuple([model_year[0] for model_year in years])

Running query in 'duckdb:///../data/database/car_data.duckdb'

In [4]:
makes = %sql select DISTINCT(make) from all_vehicles
makes = [m[0] for m in makes]

Running query in 'duckdb:///../data/database/car_data.duckdb'

In [5]:
classes = %sql select DISTINCT(vehicle_class) from all_vehicles
classes = [c[0] for c in classes]

Running query in 'duckdb:///../data/database/car_data.duckdb'

In [6]:
co2 = %sql select DISTINCT(co2_rating) from all_vehicles where co2_rating is not null
co2 = [c[0] for c in co2]
# convert to int
co2 = [int(c) for c in co2]

Running query in 'duckdb:///../data/database/car_data.duckdb'

In [7]:
vehicle_type = %sql select DISTINCT(vehicle_type) from all_vehicles
vehicle_type = [v[0] for v in vehicle_type]

Running query in 'duckdb:///../data/database/car_data.duckdb'

## Interactive Table
> Multiple values for Vehcile Class can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.

In [8]:
def select_table_electric(vehicle_type, year, vehicle_class, make, co2):
    """
    Select table based on vehicle type
    
    Parameters
    ----------
    vehicle_type : str
        Vehicle type (fuel-only, hybrid, or electric)
    year : int
        Model year
    vehicle_class : list
        Vehicle class (compact, midsize, etc.)
    make : str
        Car manufacturer
    co2 : int
        CO2 rating (higher rating suggests lower CO2 emissions)
    """
    print("Performaing query")
    query = select_table(vehicle_type, year, vehicle_class, make, co2)

    # Use JupySQL magic to execute the query
    result = %sql {{query}}

    # Convert the result to a Pandas DataFrame
    df = result.DataFrame()

    clear_output(wait=True)

    show(df, classes="display nowrap compact")

(
    widget_vehicle_type,
    widget_year,
    widget_make,
    widget_vehicle_class,
    widget_co2,
) = init_widgets(years, makes, classes, vehicle_type, style)

tab = setup_menu(
    widget_vehicle_type, widget_year, widget_vehicle_class, widget_make, widget_co2
)

output = interactive_output(
    select_table_electric,
    {
        "vehicle_type": widget_vehicle_type,
        "year": widget_year,
        "vehicle_class": widget_vehicle_class,
        "make": widget_make,
        "co2": widget_co2,
    },
)

display(tab, output)

Output()

## Plots

This dashboard contains 5 plots, created using the `ggplot` API, `seaborn`, and `ipywidgets`:

1. Bar Plot of Car Manufacturing Trends
2. Boxplot of Fuel Consumption and GHG emissions for All Vehicle Types
3. Scatter Plot of Electric Vehicle Ranges and Charging Time by Car Size and Model Year
4. Histogram of GHG emissions by Vehicle and Fuel Type
5. GHG emissions of Hybrid and Fuel-Only US Car Brands by Transmission Type

## Bar plot of car manufacturing trends